In [ ]:
# optional, only for Jupyter
%matplotlib notebook

# General libraries
import numpy as np                # to deal with arrays, vectors, matrices...
import matplotlib.pyplot as plt   # to plot the data
import matplotlib.gridspec as gridspec

# Tensorflow
import os
HOME = os.getenv('HOME')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # to get rid of the TF compilation warnings
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

In [ ]:
# Only because my system-wide config is tuned, you don't need these lines
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 5,3
mpl.rcParams['font.size'] = 12.0

## Get the data from the public MNIST library
Each sample is a $28\times28$ picture of handwritten numbers.  
The dataset will have dimension $N\times28\times28$

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# Normalize pixel values
x_train, x_test = x_train/255, x_test/255

# Fix dimensions: (Nsamples, width, height, channels)
x_train = np.expand_dims(x_train, axis=3)
y_train = np.expand_dims(y_train, axis=1)
x_test = np.expand_dims(x_test, axis=3)
y_test = np.expand_dims(y_test, axis=1)

# Useful
inp_shape = x_train.shape[1:]
Nclasses =  len(np.unique(y_train))
        
# Report dimensions
print(f'Training dataset --> images: {x_train.shape}; labels: {y_train.shape}')
print(f'Testing dataset  --> images: {x_test.shape}; labels: {y_test.shape}')
print(f'N classes: {Nclasses}')

In [ ]:
# choose a random sample
ind = np.random.randint(0,x_train.shape[0])

img = x_train[ind,:,:,0]
print(img.shape)

# Plot the sample
fig, ax = plt.subplots()
ax.imshow(img)
ax.set_title(y_train[ind])
ax.grid()
plt.show()

In [ ]:
# Build the model
model = models.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=inp_shape))
model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(Nclasses, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              #loss='sparse_categorical_crossentropy',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(), #from_logits=True),
              metrics=['accuracy'])

In [ ]:
# Before training, random accuracy
loss,acc = model.evaluate(x_test, y_test,verbose=0)
print(f'accuracy: {acc*100:.2f}%')

# Training
from time import time
t_old = time()
print('Training...')
history = model.fit(x_train, y_train, epochs=5,
                    validation_data=(x_test, y_test),
                    #validation_split=0.1,
                    verbose=1)
print('...Done in %ss'%(time()-t_old))

In [ ]:
# plot learning curve
err = history.history['loss']
val_err = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Plots grid
fig, ax = plt.subplots(figsize=(5,5))
gs = gridspec.GridSpec(2, 1)
ax0 = plt.subplot(gs[0, 0])
ax1 = plt.subplot(gs[1, 0])

# Loss plots
ax0.plot(err, label='Train')
ax0.plot(val_err, label='Test')
ax0.set_ylabel('Loss')

# Accuracy plots
ax1.plot(acc,label='Train')
ax1.plot(val_acc,label='Test')
ax1.set_ylabel('Accuracy')

# General settings
ax0.set_title('Learning curve')
plt.show()

In [ ]:
loss,acc = model.evaluate(x_test, y_test,verbose=0)
print(f'accuracy: {acc*100:.2f}%')

In [ ]:
# Show Results
predictions = model.predict(x_test)

In [ ]:
# 4 Random examples
n = 3
samples = [np.random.randint(0,x_test.shape[0]) for _ in range(n*n)]

fig, ax = plt.subplots(figsize=(5,5))
gs = gridspec.GridSpec(n, n)
axs = []
for i in range(n):
    for j in range(n):
        axs.append( plt.subplot(gs[i,j]) )

for i in range(len(samples)):
    ax = axs[i]
    ind = samples[i]
    img = x_test[ind,:,:,0]
    label = y_test[ind]
    predicted = np.argmax(predictions[ind])
    ax.imshow(img)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(1,4, str(predicted),
            bbox=dict(boxstyle="square",
                      fc='w',
                      alpha=0.5) )

fig.suptitle(f'Acc: {acc*100:.2f}%')
plt.show()

In [ ]:
model.save('mnist_cnn.h5')